# 하이퍼 파라미터 튜닝
### Trainer 클래스의 hyperparameter_search를 사용하여 튜닝하기
- 필요에 맞게 `config_data`의 `model_name`과 `data_path` 수정 (`config_data['general']['model_name]`, `config_data['general']['data_path]`)
- 튜닝하고 싶은 하이퍼파라미터와 튜닝할 범위 설정 
    - 함수 `my_hp_space`에서 변경
- 함수 `my_objective`에서 `['eval_loss', 'eval_rouge-1', 'eval_rouge-2', 'eval_rouge-l']` 중 최적화하고 싶은 것으로 변경
    - `eval_loss`의 경우 `trainer.hyperparameter_search(direction="maximize", ...)`에서  `direction='minimize'`로 수정
- `trainer.hyperparameter_search(direction="maximize", n_trial=5, ...)`에서 필요에 맞게 `n_trial` 변경
- 이외 필요에 맞게 수정

In [2]:
import os
os.environ["WANDB_DISABLED"] = "true"

In [3]:
import pandas as pd
import optuna
from rouge import Rouge

from transformers import AutoTokenizer, BartForConditionalGeneration, BartConfig
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer
from transformers import Trainer, TrainingArguments
from transformers import EarlyStoppingCallback

import torch
from torch.utils.data import Dataset , DataLoader

In [4]:
# torch.cuda.empty_cache()

In [5]:
tokenizer = AutoTokenizer.from_pretrained("digit82/kobart-summarization")

In [6]:
# 사용할 모델 이름(model_name)과 파일 경로 변경

config_data = {
    "general": {
        "data_path": "../data/", # 모델 생성에 필요한 데이터 경로를 사용자 환경에 맞게 지정합니다.
        "model_name": "digit82/kobart-summarization", # 불러올 모델의 이름을 사용자 환경에 맞게 지정할 수 있습니다.
        "output_dir": "./" # 모델의 최종 출력 값을 저장할 경로를 설정합니다.
    },
    "tokenizer": {
        "encoder_max_len": 512,
        "decoder_max_len": 100,
        "bos_token": f"{tokenizer.bos_token}",
        "eos_token": f"{tokenizer.eos_token}",
        # 특정 단어들이 분해되어 tokenization이 수행되지 않도록 special_tokens을 지정해줍니다.
        "special_tokens": ['#Address#', '#CarNumber#', '#CardNumber#', '#DateOfBirth#', '#Email#', '#PassportNumber#', 
                           '#Person#', '#Person1#', '#Person2#', '#Person3#', '#Person4#', '#Person5#', '#Person6#', 
                           '#Person7#', '#PhoneNumber#', '#SSN#']
    },
    "training": {
        "overwrite_output_dir": True,
        "num_train_epochs": 20,
        "learning_rate": 1e-5,
        "per_device_train_batch_size": 50,
        "per_device_eval_batch_size": 32,
        "warmup_ratio": 0.1,
        "weight_decay": 0.01,
        "lr_scheduler_type": 'cosine',
        "optim": 'adamw_torch',
        "gradient_accumulation_steps": 1,
        "evaluation_strategy": 'epoch',
        "save_strategy": 'epoch',
        "save_total_limit": 20,             # checkpoint 생성 개수
        "fp16": True,
        "load_best_model_at_end": True,
        "seed": 42,
        "logging_dir": "./logs",
        "logging_strategy": "epoch",
        "predict_with_generate": True,
        "generation_max_length": 100,
        "do_train": True,
        "do_eval": True,
        "early_stopping_patience": 3    ,
        "early_stopping_threshold": 0.001,
        "report_to": None
    },
    "inference": {
        "ckt_path": "model ckt path", # 사전 학습이 진행된 모델의 checkpoint를 저장할 경로를 설정합니다.
        "result_path": "./prediction/",
        "no_repeat_ngram_size": 2,
        "early_stopping": True,
        "generate_max_length": 100,
        "num_beams": 4,
        "batch_size" : 32,
        # 정확한 모델 평가를 위해 제거할 불필요한 생성 토큰들을 정의합니다.
        "remove_tokens": ['<usr>', f"{tokenizer.bos_token}", f"{tokenizer.eos_token}", f"{tokenizer.pad_token}"]
    }
}

In [7]:
# 모델 성능에 대한 평가 지표를 정의합니다. 본 대회에서는 ROUGE 점수를 통해 모델의 성능을 평가합니다.
def compute_metrics(config,tokenizer,pred):
    rouge = Rouge()
    predictions = pred.predictions
    labels = pred.label_ids

    predictions[predictions == -100] = tokenizer.pad_token_id
    labels[labels == -100] = tokenizer.pad_token_id

    decoded_preds = tokenizer.batch_decode(predictions, clean_up_tokenization_spaces=True)
    labels = tokenizer.batch_decode(labels, clean_up_tokenization_spaces=True)

    # 정확한 평가를 위해 미리 정의된 불필요한 생성토큰들을 제거합니다.
    replaced_predictions = decoded_preds.copy()
    replaced_labels = labels.copy()
    remove_tokens = config['inference']['remove_tokens']
    for token in remove_tokens:
        replaced_predictions = [sentence.replace(token," ") for sentence in replaced_predictions]
        replaced_labels = [sentence.replace(token," ") for sentence in replaced_labels]

    print('-'*150)
    print(f"PRED: {replaced_predictions[0]}")
    print(f"GOLD: {replaced_labels[0]}")
    print('-'*150)
    print(f"PRED: {replaced_predictions[1]}")
    print(f"GOLD: {replaced_labels[1]}")
    print('-'*150)
    print(f"PRED: {replaced_predictions[2]}")
    print(f"GOLD: {replaced_labels[2]}")

    # 최종적인 ROUGE 점수를 계산합니다.
    results = rouge.get_scores(replaced_predictions, replaced_labels,avg=True)

    # ROUGE 점수 중 F-1 score를 통해 평가합니다.
    result = {key: value["f"] for key, value in results.items()}
    return result

In [8]:
# 데이터 전처리를 위한 클래스로, 데이터셋을 데이터프레임으로 변환하고 인코더와 디코더의 입력을 생성합니다.
class Preprocess:
    def __init__(self,
            bos_token: str,
            eos_token: str,
        ) -> None:

        self.bos_token = bos_token
        self.eos_token = eos_token

    @staticmethod
    # 실험에 필요한 컬럼을 가져옵니다.
    def make_set_as_df(file_path, is_train = True):
        if is_train:
            df = pd.read_csv(file_path)
            train_df = df[['fname','dialogue','summary']]
            return train_df
        else:
            df = pd.read_csv(file_path)
            test_df = df[['fname','dialogue']]
            return test_df

    # BART 모델의 입력, 출력 형태를 맞추기 위해 전처리를 진행합니다.
    def make_input(self, dataset,is_test = False):
        if is_test:
            encoder_input = dataset['dialogue']
            decoder_input = [self.bos_token] * len(dataset['dialogue'])
            return encoder_input.tolist(), list(decoder_input)
        else:
            encoder_input = dataset['dialogue']
            decoder_input = dataset['summary'].apply(lambda x : self.bos_token + str(x)) # Ground truth를 디코더의 input으로 사용하여 학습합니다.
            decoder_output = dataset['summary'].apply(lambda x : str(x) + self.eos_token)
            return encoder_input.tolist(), decoder_input.tolist(), decoder_output.tolist()

In [9]:
# Train에 사용되는 Dataset 클래스를 정의합니다.
class DatasetForTrain(Dataset):
    def __init__(self, encoder_input, decoder_input, labels, len):
        self.encoder_input = encoder_input
        self.decoder_input = decoder_input
        self.labels = labels
        self.len = len

    def __getitem__(self, idx):
        item = {key: val[idx].clone().detach() for key, val in self.encoder_input.items()} # item[input_ids], item[attention_mask]
        item2 = {key: val[idx].clone().detach() for key, val in self.decoder_input.items()} # item2[input_ids], item2[attention_mask]
        item2['decoder_input_ids'] = item2['input_ids']
        item2['decoder_attention_mask'] = item2['attention_mask']
        item2.pop('input_ids')
        item2.pop('attention_mask')
        item.update(item2) #item[input_ids], item[attention_mask] item[decoder_input_ids], item[decoder_attention_mask]
        item['labels'] = self.labels['input_ids'][idx] #item[input_ids], item[attention_mask] item[decoder_input_ids], item[decoder_attention_mask], item[labels]
        return item

    def __len__(self):
        return self.len

# Validation에 사용되는 Dataset 클래스를 정의합니다.
class DatasetForVal(Dataset):
    def __init__(self, encoder_input, decoder_input, labels, len):
        self.encoder_input = encoder_input
        self.decoder_input = decoder_input
        self.labels = labels
        self.len = len

    def __getitem__(self, idx):
        item = {key: val[idx].clone().detach() for key, val in self.encoder_input.items()} # item[input_ids], item[attention_mask]
        item2 = {key: val[idx].clone().detach() for key, val in self.decoder_input.items()} # item2[input_ids], item2[attention_mask]
        item2['decoder_input_ids'] = item2['input_ids']
        item2['decoder_attention_mask'] = item2['attention_mask']
        item2.pop('input_ids')
        item2.pop('attention_mask')
        item.update(item2) #item[input_ids], item[attention_mask] item[decoder_input_ids], item[decoder_attention_mask]
        item['labels'] = self.labels['input_ids'][idx] #item[input_ids], item[attention_mask] item[decoder_input_ids], item[decoder_attention_mask], item[labels]
        return item

    def __len__(self):
        return self.len

In [10]:
# tokenization 과정까지 진행된 최종적으로 모델에 입력될 데이터를 출력합니다.
def prepare_train_dataset(config, preprocessor, data_path, tokenizer):
    train_file_path = os.path.join(data_path,'train.csv')
    val_file_path = os.path.join(data_path,'dev.csv')

    # train, validation에 대해 각각 데이터프레임을 구축합니다.
    train_data = preprocessor.make_set_as_df(train_file_path)
    val_data = preprocessor.make_set_as_df(val_file_path)

    print('-'*150)
    print(f'train_data:\n {train_data["dialogue"][0]}')
    print(f'train_label:\n {train_data["summary"][0]}')

    print('-'*150)
    print(f'val_data:\n {val_data["dialogue"][0]}')
    print(f'val_label:\n {val_data["summary"][0]}')

    encoder_input_train , decoder_input_train, decoder_output_train = preprocessor.make_input(train_data)
    encoder_input_val , decoder_input_val, decoder_output_val = preprocessor.make_input(val_data)
    print('-'*10, 'Load data complete', '-'*10,)

    tokenized_encoder_inputs = tokenizer(encoder_input_train, return_tensors="pt", padding=True,
                            add_special_tokens=True, truncation=True, max_length=config['tokenizer']['encoder_max_len'], return_token_type_ids=False)
    tokenized_decoder_inputs = tokenizer(decoder_input_train, return_tensors="pt", padding=True,
                        add_special_tokens=True, truncation=True, max_length=config['tokenizer']['decoder_max_len'], return_token_type_ids=False)
    tokenized_decoder_ouputs = tokenizer(decoder_output_train, return_tensors="pt", padding=True,
                        add_special_tokens=True, truncation=True, max_length=config['tokenizer']['decoder_max_len'], return_token_type_ids=False)

    train_inputs_dataset = DatasetForTrain(tokenized_encoder_inputs, tokenized_decoder_inputs, tokenized_decoder_ouputs,len(encoder_input_train))

    val_tokenized_encoder_inputs = tokenizer(encoder_input_val, return_tensors="pt", padding=True,
                        add_special_tokens=True, truncation=True, max_length=config['tokenizer']['encoder_max_len'], return_token_type_ids=False)
    val_tokenized_decoder_inputs = tokenizer(decoder_input_val, return_tensors="pt", padding=True,
                        add_special_tokens=True, truncation=True, max_length=config['tokenizer']['decoder_max_len'], return_token_type_ids=False)
    val_tokenized_decoder_ouputs = tokenizer(decoder_output_val, return_tensors="pt", padding=True,
                        add_special_tokens=True, truncation=True, max_length=config['tokenizer']['decoder_max_len'], return_token_type_ids=False)

    val_inputs_dataset = DatasetForVal(val_tokenized_encoder_inputs, val_tokenized_decoder_inputs, val_tokenized_decoder_ouputs,len(encoder_input_val))

    print('-'*10, 'Make dataset complete', '-'*10,)
    return train_inputs_dataset, val_inputs_dataset

In [11]:
def model_init():
    model_name = config_data["general"]["model_name"]
    bart_config = BartConfig().from_pretrained(model_name)
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    generate_model = BartForConditionalGeneration.from_pretrained(model_name,config=bart_config)

    special_tokens_dict={'additional_special_tokens':config_data['tokenizer']['special_tokens']}
    tokenizer.add_special_tokens(special_tokens_dict)

    generate_model.resize_token_embeddings(len(tokenizer)) # 사전에 special token을 추가했으므로 재구성 해줍니다.

    device = torch.device('cuda:0' if torch.cuda.is_available()  else 'cpu')
    generate_model.to(device)
    return generate_model

In [12]:
def load_tokenizer():
    model_name = config_data["general"]["model_name"]
    tokenizer = AutoTokenizer.from_pretrained(model_name)

    special_tokens_dict={'additional_special_tokens':config_data['tokenizer']['special_tokens']}
    tokenizer.add_special_tokens(special_tokens_dict)
    return tokenizer

In [13]:
preprocessor = Preprocess(config_data['tokenizer']['bos_token'], config_data['tokenizer']['eos_token'])

In [14]:
device = torch.device('cuda:0' if torch.cuda.is_available()  else 'cpu')
tokenizer = load_tokenizer()

In [15]:
data_path = config_data["general"]["data_path"]

In [16]:
train_inputs_dataset, val_inputs_dataset = prepare_train_dataset(config_data, preprocessor, data_path, tokenizer)

------------------------------------------------------------------------------------------------------------------------------------------------------
train_data:
 #Person1#: 안녕하세요, 스미스씨. 저는 호킨스 의사입니다. 오늘 왜 오셨나요?
#Person2#: 건강검진을 받는 것이 좋을 것 같아서요.
#Person1#: 그렇군요, 당신은 5년 동안 건강검진을 받지 않았습니다. 매년 받아야 합니다.
#Person2#: 알고 있습니다. 하지만 아무 문제가 없다면 왜 의사를 만나러 가야 하나요?
#Person1#: 심각한 질병을 피하는 가장 좋은 방법은 이를 조기에 발견하는 것입니다. 그러니 당신의 건강을 위해 최소한 매년 한 번은 오세요.
#Person2#: 알겠습니다.
#Person1#: 여기 보세요. 당신의 눈과 귀는 괜찮아 보입니다. 깊게 숨을 들이쉬세요. 스미스씨, 담배 피우시나요?
#Person2#: 네.
#Person1#: 당신도 알다시피, 담배는 폐암과 심장병의 주요 원인입니다. 정말로 끊으셔야 합니다. 
#Person2#: 수백 번 시도했지만, 습관을 버리는 것이 어렵습니다.
#Person1#: 우리는 도움이 될 수 있는 수업과 약물들을 제공하고 있습니다. 나가기 전에 더 많은 정보를 드리겠습니다.
#Person2#: 알겠습니다, 감사합니다, 의사선생님.
train_label:
 스미스씨가 건강검진을 받고 있고, 호킨스 의사는 매년 건강검진을 받는 것을 권장합니다. 호킨스 의사는 스미스씨가 담배를 끊는 데 도움이 될 수 있는 수업과 약물에 대한 정보를 제공할 것입니다.
------------------------------------------------------------------------------------------------------------------------------------------

---------- Make dataset complete ----------


In [17]:
# 학습을 위한 trainer 클래스와 매개변수를 정의합니다.
def load_trainer_for_train(training_args, config,tokenizer,train_inputs_dataset,val_inputs_dataset):
    # Validation loss가 더 이상 개선되지 않을 때 학습을 중단시키는 EarlyStopping 기능을 사용합니다.
    MyCallback = EarlyStoppingCallback(
        early_stopping_patience=config['training']['early_stopping_patience'],
        early_stopping_threshold=config['training']['early_stopping_threshold']
    )
    print('-'*10, 'Make training arguments complete', '-'*10,)
    print('-'*10, 'Make trainer', '-'*10,)

    # Trainer 클래스를 정의합니다.
    trainer = Seq2SeqTrainer(
        args=training_args,
        train_dataset=train_inputs_dataset,
        eval_dataset=val_inputs_dataset,
        model_init=model_init,
        compute_metrics = lambda pred: compute_metrics(config,tokenizer, pred),
        callbacks = [MyCallback]
    )
    print('-'*10, 'Make trainer complete', '-'*10,)

    return trainer

In [18]:
training_args = Seq2SeqTrainingArguments(
            output_dir=config_data['general']['output_dir'], # model output directory
            overwrite_output_dir=config_data['training']['overwrite_output_dir'],
            num_train_epochs=config_data['training']['num_train_epochs'],  # total number of training epochs
            learning_rate=config_data['training']['learning_rate'], # learning_rate
            per_device_train_batch_size=config_data['training']['per_device_train_batch_size'], # batch size per device during training
            per_device_eval_batch_size=config_data['training']['per_device_eval_batch_size'],# batch size for evaluation
            warmup_ratio=config_data['training']['warmup_ratio'],  # number of warmup steps for learning rate scheduler
            weight_decay=config_data['training']['weight_decay'],  # strength of weight decay
            lr_scheduler_type=config_data['training']['lr_scheduler_type'],
            optim =config_data['training']['optim'],
            gradient_accumulation_steps=config_data['training']['gradient_accumulation_steps'],
            evaluation_strategy=config_data['training']['evaluation_strategy'], # evaluation strategy to adopt during training
            save_strategy =config_data['training']['save_strategy'],
            save_total_limit=config_data['training']['save_total_limit'], # number of total save model.
            fp16=config_data['training']['fp16'],
            load_best_model_at_end=config_data['training']['load_best_model_at_end'], # 최종적으로 가장 높은 점수 저장
            seed=config_data['training']['seed'],
            logging_dir=config_data['training']['logging_dir'], # directory for storing logs
            logging_strategy=config_data['training']['logging_strategy'],
            predict_with_generate=config_data['training']['predict_with_generate'], #To use BLEU or ROUGE score
            generation_max_length=config_data['training']['generation_max_length'],
            do_train=config_data['training']['do_train'],
            do_eval=config_data['training']['do_eval'],
            report_to=config_data['training']['report_to'] # (선택) wandb를 사용할 때 설정합니다.
        )

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [19]:
# 튜닝하고자 하는 hyperparameter와 범위 지정

def my_hp_space(trial):
    return {
        "learning_rate": trial.suggest_float("learning_rate", low=1e-5, high=0.01), # learning_rate
        "per_device_train_batch_size" : trial.suggest_categorical("per_device_train_batch_size", [4, 8, 16, 32, 50]),
        "per_device_eval_batch_size" : trial.suggest_categorical("per_device_eval_batch_size", [4, 8, 16, 32, 50]),
        "weight_decay" : trial.suggest_float("weight_decay", 4e-5, 0.01),
        "lr_scheduler_type" : trial.suggest_categorical("lr_scheduler_type", ["linear", "cosine"]),
        "optim" : trial.suggest_categorical("optim", ["adamw_hf", "adamw_torch", "adamw_torch_fused", "adafactor"])
    }


In [20]:
def my_objective(metrics):
    return metrics["eval_rouge-1"]

In [21]:
trainer = load_trainer_for_train(training_args, config_data, tokenizer,train_inputs_dataset,val_inputs_dataset)

Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


---------- Make training arguments complete ----------
---------- Make trainer ----------


You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


---------- Make trainer complete ----------


In [22]:
# 필요에 맞게 n_trial 변경
trainer.hyperparameter_search(direction="maximize", compute_objective=my_objective, n_trials=5, hp_space = my_hp_space)

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
[I 2024-03-18 08:59:56,971] A new study created in memory with name: no-name-00a5e695-0ab2-4537-b818-dae732f039c2
Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.
You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.
/opt/conda/lib/python3.10/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,Rouge-1,Rouge-2,Rouge-l
1,1.460900,1.357787,0.272626,0.071159,0.258078
2,3.042400,2.906148,0.121920,0.000000,0.121920
3,2.847600,3.106741,0.121920,0.000000,0.121920
4,2.765400,7.355337,0.000000,0.000000,0.000000


------------------------------------------------------------------------------------------------------------------------------------------------------
PRED:  #Person2# 는 #Person1# 에게 자신의 약혼혈에 대해 이야기합니다. #Person2# 는 #Person1# 에게 약을 복용하는 것을 제안합니다.                                                                        
GOLD: #Person2# 는 숨쉬기에 어려움을 겪는다. 의사는 #Person1# 에게 이에 대해 묻고, #Person2# 를 폐 전문의에게 보낼 예정이다.                                                              
------------------------------------------------------------------------------------------------------------------------------------------------------
PRED:  #Person2# 는 #Person1# 에게 오늘 밤의 날에 대해 이야기합니다. #Person2# 는 #Person1# 에게 도움을 청합니다.                                                                           
GOLD: #Person1# 은 지미에게 운동하러 가자고 제안하고 팔과 배를 운동하도록 설득한다.                                                                        
-------------------------------------------------------------------------------------------

There were missing keys in the checkpoint model loaded: ['model.encoder.embed_tokens.weight', 'model.decoder.embed_tokens.weight', 'lm_head.weight'].
[I 2024-03-18 09:07:55,555] Trial 0 finished with value: 0.0 and parameters: {'learning_rate': 0.003969161664476582, 'per_device_train_batch_size': 50, 'per_device_eval_batch_size': 16, 'weight_decay': 0.005519606991801111, 'lr_scheduler_type': 'cosine', 'optim': 'adamw_hf'}. Best is trial 0 with value: 0.0.
Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.
You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


Epoch,Training Loss,Validation Loss,Rouge-1,Rouge-2,Rouge-l
1,1.221700,1.478840,0.290275,0.077348,0.275440


------------------------------------------------------------------------------------------------------------------------------------------------------
PRED:  #Person1# 은 #Person2# 에게 #Person2# 의 도움으로 체크인을 구매합니다.    
GOLD: #Person2# 는 숨쉬기에 어려움을 겪는다. 의사는 #Person1# 에게 이에 대해 묻고, #Person2# 를 폐 전문의에게 보낼 예정이다.                                                              
------------------------------------------------------------------------------------------------------------------------------------------------------
PRED:  #Person1# 은 #Person2# 에게 #Person2# 의 도움으로 체크인을 구매합니다.    
GOLD: #Person1# 은 지미에게 운동하러 가자고 제안하고 팔과 배를 운동하도록 설득한다.                                                                        
------------------------------------------------------------------------------------------------------------------------------------------------------
PRED:  #Person1# 은 #Person2# 에게 #Person2# 의 도움으로 체크인을 구매합니다.    
GOLD: #Person1# 은 건강에 해로운 음식을 먹는 것을 멈추려는 계획을 세우고, #Person2# 는 자신의 건강한 레시피를